In [3]:
RAW_FILE = "D:\\DungFolder\\workspace\\wiki_retrieval\\zac2022-data\\wikipedia_20220620_cleaned_v2.csv"
TOTAL_CHUNK = 1944407
BM25_FILE = "D:\\DungFolder\\workspace\\wiki_retrieval\\bm25\\collections\\wiki\\wiki.jsonl"
COLLECTION_FILE = "D:\\DungFolder\\workspace\\wiki_retrieval\\notebooks\\wiki.db"

In [ ]:
from datasets import load_dataset
collection_dataset = load_dataset('csv', data_files=RAW_FILE, streaming=True)["train"]

In [2]:
import sqlite3

class WikiDataset:
    def __init__(self, db_path='wiki.db'):
        self.conn = sqlite3.connect(db_path)
        self.cursor = self.conn.cursor()
        self._create_table()

    def _create_table(self):
        self.cursor.execute('''
            CREATE TABLE IF NOT EXISTS wiki (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT NOT NULL,
                text TEXT NOT NULL
            )
        ''')
        self.conn.commit()

    def add_row(self, index, title, text):
        self.cursor.execute('''
            INSERT INTO wiki (id, title, text) VALUES (?, ?, ?)
        ''', (index, title, text))
        self.conn.commit()

    def __getitem__(self, index):
        self.cursor.execute('''
            SELECT id, title, text FROM wiki WHERE id = ?
        ''', (index,))
        row = self.cursor.fetchone()
        if row is None:
            raise IndexError("Index out of range")
        return {'index': row[0], 'title': row[1], 'text': row[2]}

    def __len__(self):
        self.cursor.execute('SELECT COUNT(*) FROM wiki')
        return self.cursor.fetchone()[0]

    def close(self):
        self.conn.close()

wikidataset = WikiDataset(COLLECTION_FILE)

In [4]:
import jsonlines
from tqdm import tqdm

with jsonlines.open(BM25_FILE, mode="w") as writer:
    for index, item in enumerate(tqdm(collection_dataset, total=TOTAL_CHUNK)):
        wikidataset.add_row(
            index=index,
            title=item["title"],
            text=item["text"]
        )
        writer.write({
            'id': index,
            'contents': item['bm25_text']
        })
        if index > 1000:
            break

  0%|          | 1001/1944407 [00:03<1:38:40, 328.24it/s]
